# Linear Classification II - Naive Bayes Classification Homework

In [203]:
from sklearn import datasets
import pandas as pd
import numpy as np


#maybe make dataframe and get data description.
X, y, centers = datasets.make_blobs(n_samples = 400, n_features = 5, 
            centers=4, cluster_std = 2, random_state=100, return_centers=True)

DF_y= pd.DataFrame(X)
DF_y['y']= y
# DF[4] = pd.DataFrame.from_records(y)
# DF_y.describe()
# print(DF[0])

N, D = X.shape

## A. Compute the prior probability of each class, p(Ck)

As the problem statement specifies the amount of observations in each target class, we can infer that all prior probabilities are equal at 1/4. We will want to do the computation to ensure this fact. 

In [204]:
c0= 0
c1= 0
c2= 0
c3= 0

for i in y: 
    if i == 0:
        c0=c0+1
    if i == 1:
        c1=c1+1
    if i == 2: 
        c2=c2+1
    if i == 3:
        c3=c3+1
        

ck = [0, 1, 2, 3]
a = [c0,c1,c2,c3]
ct = 0

priors = []
for i in a:
    priors.append(i/N)

for i in priors: 
    print("prior p(c" + str(ck[ct]) + "): " + str(i))
    ct = ct +1
    

prior p(c0): 0.25
prior p(c1): 0.25
prior p(c2): 0.25
prior p(c3): 0.25


## B. Compute the likelihood p(X|Ck)

As naive bayes assumes that our classes are independent although they may not be, we can simplify the likelihood 
P(X0, X1, X2, X3 | Y) to just i=1 ∏ D p(Xi | Y). For this, we must find the distribution (pdf~ probability density function) p(feature(i) | Y) accounting for target correlation using μk and σk^(2) where k denotes the number of classes and the belonging class. In other words we must find the distribution for each feature that maps to each target, and we should expect 4 features x 4 classes = 16 values in the final likelihood calculation.

In [205]:
feats = [0,1,2,3,4]
means = DF_y.groupby('y')[feats].mean()
variances = DF_y.groupby('y')[feats].var()

means = means.to_numpy()
variances = variances.to_numpy()

In addition, it will be crucial to draw normal distributions coming from each feature given each class (target value). With the means and variances computed, we use the probability density function.

In [206]:
def pdf(mean, variance, xi):
    t1= 1/(variance*np.sqrt(2*np.pi))
    return t1*np.exp(-((xi - mean) ** 2) / (2 * variance))

In [207]:
likelihood= np.zeros((N, len(ck))) ## N X len(y) likelihood matrix

for i in range(len(ck)): ## i ~ classes = 0 -> 3
    likelihoods = np.zeros(N)
    for x in range(D): ## x ~ features = 0 -> 4
        likelihoods+=np.log(pdf(means[i, x], variances[i, x], X[:, x]))
    likelihood[:, i] = likelihoods

likelihood

array([[-45.6985698 , -84.05970182, -55.50213622, -13.08991735],
       [-43.67078578, -68.5409193 , -18.30734434, -71.81973225],
       [-12.54998639, -67.10163115, -54.08446763, -45.33746018],
       ...,
       [-46.42621433, -69.91300807, -47.59980544, -14.30066169],
       [-41.37699859, -66.05509473, -12.0967166 , -50.50308082],
       [-38.15476588, -62.83221841, -37.61153955, -15.48573157]])

## C. Compute the posterior probability of each point p(Ck|X). Assign the class ID to each point

We understand that the posterior probability is proportional to the likelihood times the prior probabilities. By using np.argmax, we assign classes or labels to each observation in predicting its designated class.

In [208]:
posterior = likelihood*priors
# posterior

predicted_classes = np.argmax(posterior, axis=1) #attach labels...
predicted_classes

array([3, 2, 0, 3, 3, 1, 3, 1, 2, 2, 0, 2, 2, 3, 0, 3, 2, 3, 1, 2, 2, 1,
       1, 3, 1, 3, 1, 0, 2, 0, 2, 3, 0, 0, 3, 0, 1, 1, 1, 2, 1, 3, 2, 0,
       0, 3, 3, 2, 3, 1, 3, 1, 2, 2, 3, 1, 0, 1, 3, 2, 2, 0, 3, 1, 1, 0,
       3, 2, 1, 0, 1, 2, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1, 0, 0, 3,
       2, 1, 1, 3, 3, 0, 2, 1, 3, 2, 3, 1, 2, 2, 1, 1, 1, 1, 2, 3, 2, 3,
       1, 3, 0, 3, 2, 2, 1, 1, 1, 3, 1, 3, 0, 3, 0, 2, 1, 3, 3, 2, 3, 2,
       3, 0, 2, 2, 2, 3, 1, 3, 2, 0, 0, 0, 0, 2, 3, 1, 3, 0, 0, 1, 3, 1,
       2, 0, 0, 3, 0, 3, 1, 0, 0, 2, 0, 3, 1, 2, 2, 3, 0, 0, 3, 1, 1, 0,
       3, 0, 0, 0, 1, 0, 1, 0, 1, 0, 3, 2, 1, 2, 2, 0, 1, 3, 2, 3, 3, 0,
       3, 2, 0, 0, 1, 3, 0, 3, 3, 2, 1, 3, 0, 1, 2, 2, 3, 1, 2, 1, 2, 3,
       1, 0, 1, 3, 1, 0, 0, 1, 0, 0, 3, 0, 1, 0, 3, 1, 3, 3, 0, 0, 1, 3,
       0, 0, 2, 1, 2, 2, 0, 3, 0, 0, 1, 3, 3, 2, 0, 0, 1, 2, 2, 1, 3, 0,
       2, 1, 3, 3, 2, 1, 0, 2, 0, 0, 3, 3, 2, 0, 3, 2, 2, 3, 2, 2, 1, 0,
       3, 0, 1, 2, 0, 3, 1, 1, 2, 2, 0, 0, 2, 3, 1,

## D. Construct the confusion matrix to show the classification rate

In [211]:
confusion_matrix = np.zeros((len(ck), len(ck)), dtype=int)
for t, p in zip(y, predicted_classes):
    confusion_matrix[t, p] += 1

confusion_matrix/100

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

We can be certain that all points are accurately classified, seeing as the classification rate is 100% for all classes, meaning the model accurately classified the 100 points that each class began with.

## E. Classify the target using sklearn.native bayes.GaussianNB. Report the accuracy of the model

In [210]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
model = GaussianNB()
model.fit(X, y)
y_hat = model.predict(X)
accuracy = accuracy_score(y, y_hat)
accuracy


1.0

We can see that both my model and the model implemented by SciKit learn have identical performance at 100% accuracy.